In [ ]:
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
poverty = pd.read_csv('../../data/poverty.csv',low_memory=False)
df = poverty[poverty['is_country'] & poverty['year'].eq(2015)]
gini = 'GINI index (World Bank estimate)'
df = poverty[poverty['is_country'] & poverty['year'].isin([2010, 2015])]

In [ ]:
%%capture
app = JupyterDash(__name__)

app.layout = html.Div([
            dbc.Label('Indicator:'),
            dcc.Dropdown(id='hist_indicator_dropdown',
                         value='GINI index (World Bank estimate)',
                         options=[{'label': indicator, 'value': indicator}
                                  for indicator in poverty.columns[3:54]]),
            dbc.Label('Years:'),
            dcc.Dropdown(id='hist_multi_year_selector',
                         multi=True,
                         placeholder='Select one or more years',
                         options=[{'label': year, 'value': year}
                                  for year in poverty['year'].drop_duplicates().sort_values()]),
            dbc.Label('Modify number of bins:'),
            dcc.Slider(id='hist_bins_slider', 
                       dots=True, min=0, max=100, step=5, included=False,
                       marks={x: str(x) for x in range(0, 105, 5)}),
    dcc.Graph(id='indicator_year_histogram'),
])
app.run_server(mode='inline', port=3000,host='0.0.0.0')

In [ ]:
! echo 'Your app is running on: '$EDUCATIVE_LIVE_VM_URL':3000'